# Data Engineering Lab
---

In this lab we are manipulating data from several sources (local files, API, PostgreSQL) to build an analytics database.
Some actions can be ad-hoc (like the initial ingestions in this notebook) and others are recurrent ETL (Extract-Transform-Load) processes, like the data pipelines we will also go through later.


# 0 - Libraries and utilities

In [1]:
import requests
import json
import os

import pandas as pd
import sqlite3

> ⚠️ In the code below we are declaring a password and other sensitive details as readable strings. **Definitely not a good practice!!** If you ever have to do this in a production system or somewhere where your code is exposed (e.g. Github), there are several libraries that help to hide your credetials. Here are [some examples](http://theautomatic.net/2020/04/28/how-to-hide-a-password-in-a-python-script/).

In [2]:
# DB config
DB_NAME='dsa.db'

# File paths
RAW_DATA_FOLDER_PATH = 'raw_data'

# Create folder for data if it doesn't exist
if RAW_DATA_FOLDER_PATH not in os.listdir():
    os.mkdir(RAW_DATA_FOLDER_PATH)

# Table names
TEST_PREFIX=''
COVID_SURVEY_TABLE_NAME = f'{TEST_PREFIX}rpl_covid_survey'
MOVE_RANGE_TABLE_NAME = f'{TEST_PREFIX}rpl_move_range'

# Time range (for COVID survey API)
FROM_DATE = '2021-01-01'
TO_DATE = '2021-09-01'

# Other parameters
SURVEY_INDICATORS = [
    'mask',
    'covid',
    'tested_positive_14d',
    'covid_vaccine',
]

## 0.1 functions
The functions below are wrappers to interact with the database (create tables, query data, insert data from file...)

In [3]:
# Returns objects to interact with database
def connect_db():
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()
    return conn, cur

# Creates empty table in database
def create_table(table_name, table_schema, drop_if_exists=False):
    conn, cur = connect_db()
    cols = ',\n'.join([f'{col_name} {col_type}' for col_name, col_type in table_schema])
    
    if drop_if_exists:
        sql = f'DROP TABLE IF EXISTS {table_name};'
        print(sql)
        print('\n\n')
        cur.execute(sql)

    sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name}(
            {cols}
        );
    """
    print(sql)
    print('\n\n')
    cur.execute(sql)
    conn.commit()
    conn.close()


#     print('-----------------------\nFile loading: FINISHED\n-----------------------\n')
def load_file_in_table(
    file_path,
    table_name,
    table_schema=None,
    sep='\t',
    skip_header=True,
    file_encoding='utf-8',
    run_create_table=False,
    overwrite_filter=None,
):
    conn, cur = connect_db()
    
    if run_create_table:
        create_table(table_name, table_schema)
    
    if overwrite_filter:
        sql = f'DELETE FROM {table_name} WHERE {overwrite_filter}'
        print(sql)
        cur.execute(sql)
        print('\n\n')

    print('-----------------------\nFile loading: STARTED\n-----------------------\n')
    insert_statement = f"INSERT INTO {table_name} ({', '.join([col[0] for col in table_schema])}) VALUES({', '.join(['?' for col in table_schema])})"
    print(insert_statement)
    df = pd.read_csv(file_path, sep=sep)
    df.to_sql(table_name, con=conn, if_exists='append', index=False)
    conn.close()
    print('-----------------------\nFile loading: FINISHED\n-----------------------\n')


# Runs query on database
def query_db(query):
    conn, cur = connect_db()
    output = pd.io.sql.read_sql_query(query, conn)
    conn.close()
    return output

# Returns a dataframe listing all tables in database
def get_tables_list():
    return query_db("SELECT name FROM sqlite_master WHERE type='table'")

# Returns True if table exists, False otherwise
def table_exists(table_name):
    return table_name in set(get_tables_list()["name"])

# Runs query, returns nothing
def run_query(query, verbose=False):
    conn, cur = connect_db()
    if verbose:
        print(query)
    cur.execute(query)
    conn.commit()
    conn.close()

# #########################
# Other auxiliary functions
# #########################

def get_indicator_code(indicator):
    indicator_to_code = {
#         'covid': 'cli',
        'flu': 'ili',
        'mask': 'mc',
        'contact': 'dc',
        'finance': 'hf',
        'anosmia': 'anos',
        'vaccine_acpt': 'vu',
    }
    return indicator_to_code.get(indicator, indicator)

def get_pct_prefix(indicator):
    pct_indicators = [
        "covid",
        "tested_positive_14d",
        "covid_vaccine",
    ]
    if indicator in pct_indicators:
        return "pct"
    else:
        return "percent"

Now we can query the database to confirm the data is there.
The auxiliary function ```query_db()``` gets a SQL query and returns a *pandas.DataFrame* with the output.

# 1 - Load data: COVID survey (API)
API documentation: [https://gisumd.github.io/COVID-19-API-Documentation](https://gisumd.github.io/COVID-19-API-Documentation)

Now instead of loading from a file already downloaded, we are going to connect to an API, save the output locally and then load that API response into the database.

This API can return surveys with several indicators (e.g. % people using mask, % of people recently diagnosed with COVID-19, etc) for different countries.

First we will ask the API for a list of all countries, and then retrieve several reports.


## 1.1 - Load country & regions

Using one of the auxiliary functions defined above (```load_file_in_table()```) we load the file into a table in the PostgreSQL database.

To create a table in the database we need to specify its schema (i.e. their columns and [data types](https://www.postgresql.org/docs/9.5/datatype.html)).

In [4]:
get_tables_list()

,name
0,country_regions
1,rpl_covid_survey_mask
2,rpl_covid_survey_covid
3,rpl_covid_survey_covid_vaccine


In [5]:
# request data from api
response = requests.get("https://covidmap.umd.edu/api/region").text
print("API call finished")

# convert json data to dictionary
response_dict = json.loads(response)

# convert to pandas dataframe
df = pd.DataFrame.from_dict(response_dict['data'])

# Save dataframe in file
COUNTRY_REGION_FILE_PATH = os.path.join(RAW_DATA_FOLDER_PATH , 'country_regions.txt')
df.to_csv(COUNTRY_REGION_FILE_PATH, sep='\t', index=False, encoding='utf-8')

# Load file in database
table_schema = (
    ('country', 'text'),
    ('region', 'text')
)
load_file_in_table(COUNTRY_REGION_FILE_PATH, 'country_regions', table_schema=table_schema, run_create_table=True)

API call finished

        CREATE TABLE IF NOT EXISTS country_regions(
            country text,
region text
        );
    



-----------------------
File loading: STARTED
-----------------------

INSERT INTO country_regions (country, region) VALUES(?, ?)
-----------------------
File loading: FINISHED
-----------------------



In [6]:
query_db("SELECT * FROM country_regions")

,country,region
0,Albania,Tiranë
1,Algeria,Alger
2,Algeria,Annaba
3,Algeria,Batna
4,Algeria,Béjaïa
...,...,...
2541,Vietnam,Tuyên Quang
2542,Vietnam,Vĩnh Phúc
2543,Vietnam,Yên Bái
2544,Yemen,Aden


## 1.2 - Load survey data

### 1.2.0 API call example

#### 1st sample: % of mask usage in Nigeria

In [7]:
# request data from api
response = requests.get("https://covidmap.umd.edu/api/resources?indicator=mask&type=daily&country=Nigeria&daterange=20201201-20201204").text

#convert json data to dic data for use!
response_dict = json.loads(response)

# convert to pandas dataframe
df = pd.DataFrame.from_dict(response_dict['data'])

In [8]:
df

,percent_mc,mc_se,percent_mc_unw,mc_se_unw,sample_size,country,iso_code,gid_0,survey_date
0,0.576779,0.035234,0.579634,0.025223,383.0,Nigeria,NGA,NGA,20201201
1,0.497795,0.032746,0.517157,0.024739,408.0,Nigeria,NGA,NGA,20201202
2,0.491770,0.033392,0.541885,0.025492,382.0,Nigeria,NGA,NGA,20201203
3,0.512306,0.032774,0.566434,0.023926,429.0,Nigeria,NGA,NGA,20201204


#### 2nd sample: % of people avoiding contact in Finland

In [9]:
# request data from api
response = requests.get("https://covidmap.umd.edu/api/resources?indicator=avoid_contact&type=daily&country=Finland&daterange=20201201-20201204").text

#convert json data to dic data for use!
response_dict = json.loads(response)

# convert to pandas dataframe
df = pd.DataFrame.from_dict(response_dict['data'])
df

,pct_avoid_contact,avoid_contact_se,pct_avoid_contact_unw,avoid_contact_se_unw,sample_size,country,iso_code,gid_0,survey_date
0,0.717799,0.027852,0.706977,0.021949,430.0,Finland,FIN,FIN,20201201
1,0.697544,0.031890,0.695238,0.022461,420.0,Finland,FIN,FIN,20201202
2,0.733249,0.030002,0.746341,0.021488,410.0,Finland,FIN,FIN,20201203
3,0.680975,0.031550,0.711618,0.020634,482.0,Finland,FIN,FIN,20201204


### 1.2.1 Get survey data for several indicators and countries

Using what we have seen so far, plus some loops and parameterization, we can download reports for different indicators and countries.

In [10]:
for indicator in SURVEY_INDICATORS:
    print('--------------------------\n')
    print(f'-- INDICATOR: {indicator}')
    print('\n--------------------------\n')
    
    # ############
    # Create table
    # ############
    indicator_code = get_indicator_code(indicator)
    indicator_table_name = f'{COVID_SURVEY_TABLE_NAME}_{indicator}'
    print("INDICATOR CODE: ", indicator_code)

    pct = get_pct_prefix(indicator)
    table_schema = (
        (f'{pct}_{indicator_code}', 'float'),
        (f'{indicator_code}_se', 'float'),
        (f'{pct}_{indicator_code}_unw', 'float'),
        (f'{indicator_code}_se_unw', 'float'),
        ('sample_size', 'NUMERIC'),
        ('country', 'text'),
        ('iso_code', 'text'),
        ('gid_0', 'text'),
        ('survey_date', 'NUMERIC'),
    )
    create_table(indicator_table_name, table_schema, drop_if_exists=True)

    # ####################
    # Load data into table
    # ####################
    from_date_no_dash = FROM_DATE.replace('-', '')
    to_date_no_dash = TO_DATE.replace('-', '')

    countries = [
        'Spain',
        'Japan',
        'Nigeria',
    ]

    for country in countries:
        file_path = f'{RAW_DATA_FOLDER_PATH}/covid_survey__{country}__{indicator}__{from_date_no_dash}__{to_date_no_dash}.txt'
        api_url = f"https://covidmap.umd.edu/api/resources?indicator={indicator}&type=daily&country={country}&daterange={from_date_no_dash}-{to_date_no_dash}"
        print("CALLING API: ", api_url)
        response = requests.get(api_url).text
        response_dict = json.loads(response)
        df = pd.DataFrame.from_dict(response_dict['data'])
        df.to_csv(file_path, sep='\t', index=False, encoding='utf-8')

        overwrite_filter_sql = f"country='{country}' AND survey_date BETWEEN {from_date_no_dash} AND {to_date_no_dash}"
        load_file_in_table(file_path, indicator_table_name, table_schema, overwrite_filter=overwrite_filter_sql)


--------------------------

-- INDICATOR: mask

--------------------------

INDICATOR CODE:  mc
DROP TABLE IF EXISTS rpl_covid_survey_mask;




        CREATE TABLE IF NOT EXISTS rpl_covid_survey_mask(
            percent_mc float,
mc_se float,
percent_mc_unw float,
mc_se_unw float,
sample_size NUMERIC,
country text,
iso_code text,
gid_0 text,
survey_date NUMERIC
        );
    



CALLING API:  https://covidmap.umd.edu/api/resources?indicator=mask&type=daily&country=Spain&daterange=20210101-20210901
DELETE FROM rpl_covid_survey_mask WHERE country='Spain' AND survey_date BETWEEN 20210101 AND 20210901



-----------------------
File loading: STARTED
-----------------------

INSERT INTO rpl_covid_survey_mask (percent_mc, mc_se, percent_mc_unw, mc_se_unw, sample_size, country, iso_code, gid_0, survey_date) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)
-----------------------
File loading: FINISHED
-----------------------

CALLING API:  https://covidmap.umd.edu/api/resources?indicator=mask&type=daily&

DELETE FROM rpl_covid_survey_covid_vaccine WHERE country='Nigeria' AND survey_date BETWEEN 20210101 AND 20210901



-----------------------
File loading: STARTED
-----------------------

INSERT INTO rpl_covid_survey_covid_vaccine (pct_covid_vaccine, covid_vaccine_se, pct_covid_vaccine_unw, covid_vaccine_se_unw, sample_size, country, iso_code, gid_0, survey_date) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)
-----------------------
File loading: FINISHED
-----------------------



Let's take a look at some of the data we have loaded into the db by running a very simple query:

In [11]:
query_db(f"SELECT * FROM {COVID_SURVEY_TABLE_NAME}_mask LIMIT 10")

,percent_mc,mc_se,percent_mc_unw,mc_se_unw,sample_size,country,iso_code,gid_0,survey_date
0,0.976870,0.003171,0.978026,0.002738,2867,Spain,ESP,ESP,20210101
1,0.971886,0.003932,0.975384,0.002771,3128,Spain,ESP,ESP,20210102
2,0.968848,0.004155,0.974099,0.002665,3552,Spain,ESP,ESP,20210103
3,0.962965,0.004469,0.969561,0.002968,3351,Spain,ESP,ESP,20210104
4,0.963617,0.007143,0.975008,0.002742,3241,Spain,ESP,ESP,20210105
5,0.969150,0.004062,0.974103,0.002806,3205,Spain,ESP,ESP,20210106
6,0.971114,0.003742,0.973653,0.002771,3340,Spain,ESP,ESP,20210107
7,0.969259,0.003668,0.971892,0.002814,3451,Spain,ESP,ESP,20210108
8,0.972043,0.003563,0.972437,0.002881,3229,Spain,ESP,ESP,20210109
9,0.966202,0.004061,0.970404,0.002859,3514,Spain,ESP,ESP,20210110


With the auxiliary function ```get_tables_list()``` we can take a look at all the tables in the database, created throuhout this lab.

In [12]:
get_tables_list()

,name
0,country_regions
1,rpl_covid_survey_mask
2,rpl_covid_survey_covid
3,rpl_covid_survey_tested_positive_14d
4,rpl_covid_survey_covid_vaccine


# What next?
1. In the example above we are getting the reports from a few countries. What if we want to add some more. What if we want to add them all?
2. Take a look at the entire list of indicators for the COVID survey in the [API documentation](https://gisumd.github.io/COVID-19-API-Documentation/docs/indicators/indicators.html). Would you like to include any of this in your analysis?

# 2. Data Pipeline with Luigi
[Luigi](https://luigi.readthedocs.io/en/stable/index.html) is a Python framework to schedule and orquestrate batch jobs. This framework and other similar (e.g. Apache AirFlow) are very useful to build data pipelines.

In Luigi, pipeline consists in a collection of tasks, each of them can have several funcionalities/methods. The main ones are:
- **Task.requires()**: Other tasks which are dependencies, therefore need to run before.
- **Task.run()**: The action the specific task executes/handles.
- **Task.output()**: The expected outcome for this task. In Luigi jargon this outputs are called "targets" and they can take many formats (e.g. a text file, a data partition on Hive, etc). A task is not considered complete until the target exists. If at the time of scheduling the pipeline the target exists already, then the task will not run.


## 2.1 rpl_covid_survey.py


In the file **rpl_covid_survey.py** we'll show an example of a Luigi pipeline, where we download a daily report from the API and load it into PostgreSQL. It has 4 classes:
1. CreateTable: Creates table in database, if it doesn't exist already.
2. DownloadAPIReport: Downloads report from API.
3. LoadReportIntoDB: Loads report into the database
4. MasterTask: This class generates several instances of the previous tasks, to download and ingest data for several survey indicators and countries.


To run a luigi pipeline from the terminal the code is as follows:

> python -m luigi --module {pipeline_name} {task_name} --{parameter} {parameter_value} --local-scheduler

So for our specific example it can be as follows:

> python -m luigi --module pipelines.rpl_covid_survey MasterTask --date 2021-07-01 --test-prefix test_1_ --local-scheduler
(you can omit ```--test-prefix``` parameter to insert into the "production" table instead).

## 2.1 covid_survey_covid_mask
In this pipeline we cross the data from 2 reports from 2 different indicators (*covid* and *mask*).

> python -m luigi --module pipelines.covid_survey_covid_mask LoadTable --date 2021-07-01 --test-prefix test_1_ --local-scheduler

# 3. What next?
Try writing a similar pipeline to covid_survey_covid_mask. Using SQL is what a data engineer would use in a "real scenario" querying big loads of data, but you can use Pandas if you prefer.


# 4. Homework
The COVID survey provides with a collection of indicators wide enough to run interesting analysis. But this could be crossed with other data sources to run even more insightful assessments, such as: 

- [Movement Range Maps](https://data.humdata.org/dataset/movement-range-maps)
- [HRSL (High Resolution Settlement Layer)](https://research.fb.com/downloads/high-resolution-settlement-layer-hrsl/)